In [ ]:
import numpy as np
import pandas as pd

import torch
from torch import nn,functional
from torch.autograd import Variable
import cv2 as cv


import torchvision
import torchvision.transforms as transforms

import matplotlib
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
location=''

train=pd.read_csv(location+'mnist_train.csv')[:200]
X_train=torch.from_numpy(train.loc[:,train.columns != "label"].values/255)
Y_train=torch.from_numpy(train.label.values).type(torch.LongTensor)

test=pd.read_csv(location+'mnist_test.csv')
X_test=torch.from_numpy(test.loc[:,train.columns != "label"].values/255)
Y_test=torch.from_numpy(test.label.values).type(torch.LongTensor)

train = torch.utils.data.TensorDataset(X_train,Y_train)
test = torch.utils.data.TensorDataset(X_test,Y_test)

train_loader = torch.utils.data.DataLoader(train, batch_size = 10, shuffle = True)
test_loader = torch.utils.data.DataLoader(test, batch_size = 200, shuffle = True)

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.c1=nn.Conv2d(in_channels=1, out_channels=8, stride=2, kernel_size=4, padding=1)
        self.l1=nn.Linear(288, 10)
        
        self.dropout=nn.Dropout(p=0.2)
        self.maxpool=nn.MaxPool2d(kernel_size=3, stride=2)
        self.relu=nn.ReLU()
        
    def forward(self, x):
        x=self.c1(x)
        x=self.relu(x)
        x=self.maxpool(x)

        x=x.view(x.size(0), -1)

        x=self.l1(x)
        
        return x

In [5]:
model=CNN()
model=model.double()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [6]:
epochs = 10
train_losses, test_losses, accuracy_graph = [], [], []
for epoch in range(epochs):
    running_loss = 0
    for images,labels in train_loader:
        train = Variable(images.view(-1,1,28,28))
        labels = Variable(labels)
        
        optimizer.zero_grad()
        
        output = model(train)
        loss = criterion(output,labels)

        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    else:
        test_loss = 0
        accuracy = 0
        
        with torch.no_grad(): #Turning off gradients to speed up
            model.eval()
            for images,labels in test_loader:
                
                test = Variable(images.view(-1,1,28,28))
                labels = Variable(labels)
                
                log_ps = model(test)
                test_loss += criterion(log_ps,labels)
                
                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim = 1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
        model.train()        
        train_losses.append(running_loss/len(train_loader))
        test_losses.append(test_loss/len(test_loader))
        accuracy_graph.append(accuracy/len(test_loader))

        print("Epoch: {}/{}.. ".format(epoch+1, epochs),
              "Training Loss: {:.3f}.. ".format(running_loss/len(train_loader)),
              "Test Loss: {:.3f}.. ".format(test_loss/len(test_loader)),
              "Test Accuracy: {:.3f}".format(accuracy/len(test_loader)))

Epoch: 1/10..  Training Loss: 2.296..  Test Loss: 2.317..  Test Accuracy: 0.118
Epoch: 2/10..  Training Loss: 2.277..  Test Loss: 2.305..  Test Accuracy: 0.119
Epoch: 3/10..  Training Loss: 2.262..  Test Loss: 2.294..  Test Accuracy: 0.144
Epoch: 4/10..  Training Loss: 2.248..  Test Loss: 2.281..  Test Accuracy: 0.166
Epoch: 5/10..  Training Loss: 2.233..  Test Loss: 2.270..  Test Accuracy: 0.191
Epoch: 6/10..  Training Loss: 2.220..  Test Loss: 2.261..  Test Accuracy: 0.206
Epoch: 7/10..  Training Loss: 2.205..  Test Loss: 2.249..  Test Accuracy: 0.238
Epoch: 8/10..  Training Loss: 2.191..  Test Loss: 2.238..  Test Accuracy: 0.253
Epoch: 9/10..  Training Loss: 2.176..  Test Loss: 2.226..  Test Accuracy: 0.279
Epoch: 10/10..  Training Loss: 2.161..  Test Loss: 2.213..  Test Accuracy: 0.302


In [ ]:
sb.lineplot(train_losses, label='Training loss')
sb.lineplot(test_losses, label='Validation loss')
sb.lineplot(accuracy_graph, label='Accuracy')
sb.legend(frameon=False)

C:\Users\18moh\anaconda3\lib\site-packages\seaborn\_decorators.py:36: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(


In [ ]:
class Adv(nn.Module):
    def __init__(self):
        super(Adv, self).__init__()

        self.c1=nn.Conv2d(in_channels=1, out_channels=8, stride=2, kernel_size=4, padding=1)
        self.l1=nn.Linear(288+1, 512)
        self.l2=nn.Linear(512, 784)

        self.dropout=nn.Dropout(p=0.2)
        self.maxpool=nn.MaxPool2d(kernel_size=3, stride=2)
        self.relu=nn.ReLU()
        self.sigmoid=nn.Sigmoid()

        
    def forward(self, x):
        x=self.c1(x)
        x=self.relu(x)
        x=self.maxpool(x)

        x=x.view(x.size(0), -1)
        r=torch.randn(x.size(0), 1)
        x=torch.cat((x,r), 1)

        x=self.l1(x)
        x=self.dropout(x)
        x=self.l2(x)
        x=self.relu(x)
        x=self.sigmoid(x)

        return x

In [ ]:
#loss = number of activated neurons in final layer - difference between label and classification

adv=Adv()
adv=adv.double()

cnn=CNN()
cnn=cnn.double()

criterion = nn.CrossEntropyLoss()
optCNN = torch.optim.Adam(cnn.parameters(), lr=0.0001)

optAdv = torch.optim.Adam(adv.parameters(), lr=0.0001)

In [ ]:
epochs = 1500
train_losses2, test_losses2, accuracy_graph2 = [], [], []

sample_images=[]
sample_masked=[]

for epoch in range(epochs):
    running_loss = 0
    mask_loss = 0

    currentdropout = 0.95
    
    if(epoch>400):
        currentdropout=0.75
    else:
        currentdropout = 0.95 - epoch*(0.15/400)

    i=0

    for images,labels in train_loader:
        train = Variable(images.view(-1,1,28,28))
        labels = Variable(labels)

        if(i==0):
            sample_images.append(train.clone().detach().numpy())

        optAdv.zero_grad()
        optCNN.zero_grad()

        mask = adv(train)
        finaldrop = nn.Dropout(p=currentdropout)
        mask = finaldrop(mask)

        images = images + mask
        images = torch.minimum(images, torch.ones(images.shape))
        train = Variable(images.view(-1,1,28,28))

        if(i==0):
            sample_masked.append(train.clone().detach().numpy())

        output = cnn(train)

        lossAdv = -torch.log(criterion(output, labels))
        lossCNN = criterion(output, labels)
        
        # print(lossAdv)
        # print(lossCNN)

        if(i%3==0):
            lossAdv.backward()
            optAdv.step()
        else:
            lossCNN.backward()
            optCNN.step()
        
        mask_loss += lossAdv.item()
        running_loss += lossCNN.item()
        i+=1

    else:
        test_loss = 0
        accuracy = 0
        
        with torch.no_grad(): #Turning off gradients to speed up
            cnn.eval()
            for images,labels in test_loader:
                
                test = Variable(images.view(-1,1,28,28))
                labels = Variable(labels)
                
                log_ps = cnn(test)
                test_loss += criterion(log_ps,labels)
                
                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim = 1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
        cnn.train()        
        train_losses2.append(running_loss/len(train_loader))
        test_losses2.append(test_loss/len(test_loader))
        accuracy_graph2.append(accuracy/len(test_loader))

        print("Epoch: {}/{}.. ".format(epoch+1, epochs),
              "Training Loss: {:.3f}.. ".format(running_loss/len(train_loader)),
              "Mask Loss: {:.3f}.. ".format(running_loss/len(train_loader)),
              "Test Loss: {:.3f}.. ".format(test_loss/len(test_loader)),
              "Test Accuracy: {:.3f}".format(accuracy/len(test_loader)))

In [ ]:
plt.plot(train_losses2, label='Training loss')
plt.plot(test_losses2, label='Validation loss')
plt.plot(accuracy_graph2, label='Accuracy')
plt.legend(frameon=False)

In [ ]:
plt.plot(train_losses2, label='AdvCNNTraining loss')
plt.plot(test_losses2, label='AdvCNN Validation loss')
plt.plot(accuracy_graph2, label='AdvCNN Accuracy')
plt.plot(train_losses, label='CNN Training loss')
plt.plot(test_losses, label='CNN Validation loss')
plt.plot(accuracy_graph, label='CNN Accuracy')
plt.legend(frameon=False)

In [ ]:
# Display image next to mask

epoch = 100
image = 5

f, axarr = plt.subplots(2)

axarr[0].imshow(sample_images[epoch][image][0])
axarr[1].imshow(sample_masked[epoch][image][0])

In [ ]:
3b

In [ ]:
a